In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os import walk, sep


from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import row, column
from bokeh.palettes import Dark2_5 as palette
from IPython.display import display, Markdown
import itertools
output_notebook()

Loading BokehJS ...

In [6]:
SOURCE_DIR = 'results_net_1_B_MDC_registered'

In [7]:
dfs = {}
for (dirpath, dirname, filenames) in walk(SOURCE_DIR):
    for filename in filenames:
        if filename == "metrics.csv":
            name = dirpath.split(sep)[-1]
            dfs[name] = pd.read_csv(sep.join([dirpath, filename]))

for key, df in dfs.items():
    print(f'---{key}---')
    print(df.describe())
    print()

---P25---
       Unnamed: 0        DCI         TI         Em        Bpn
count   85.000000  85.000000  85.000000  85.000000  85.000000
mean    42.000000   0.976821   0.901602   0.098398   0.002241
std     24.681302   0.006919   0.018485   0.018485   0.030029
min      0.000000   0.959172   0.854719   0.055682  -0.071815
25%     21.000000   0.970651   0.888726   0.089129  -0.021240
50%     42.000000   0.979231   0.898130   0.101870   0.007597
75%     63.000000   0.982604   0.910871   0.111274   0.024770
max     84.000000   0.987751   0.944318   0.145281   0.063481

---P21---
       Unnamed: 0        DCI         TI         Em        Bpn
count   46.000000  46.000000  46.000000  46.000000  46.000000
mean    22.500000   0.945179   0.669464   0.330536  -0.098527
std     13.422618   0.016189   0.083125   0.083125   0.130547
min      0.000000   0.891183   0.542935   0.136768  -0.409006
25%     11.250000   0.937027   0.592952   0.276530  -0.168086
50%     22.500000   0.947012   0.661510   0.33849

In [28]:
p = figure()
p.title = "P18"

In [50]:
for name, df in dfs.items():
    t = df.iloc[:,1:]
    colors = itertools.cycle(palette) 
    p = figure(x_range=t['Filename'])
    p.title = name
    x = [x for x in range(len(t['Filename'])) ]

    for feat, color in zip(t.iloc[:, 1:], colors):
        p.line(t['Filename'], t[feat], line_width=2, color=color, legend_label=f"{feat} {name}")
        p.legend.location = "top_left"
        p.legend.click_policy="mute"
        p.xaxis.major_label_orientation = "vertical"
    show(p)

In [71]:
t = dfs['P18'].describe().transpose()['mean'].transpose()

p = figure()
p.title = name
p.vbar(x=[0,1,2,3], top=[t['DCI'],20,10,20],width=0.9, color='red')
p.vbar(x=[0,1,2,3], top=[t['TI'],20,10,20],width=0.9)
show(p)
# p.legend.location = "top_left"
# p.legend.click_policy="mute"
# p.xaxis.major_label_orientation = "vertical"

In [ ]:
for name, df in dfs.items():
    t = df.iloc[:,1:]
    colors = itertools.cycle(palette) 
    p = figure(x_range=t['Filename'])
    p.title = name
    x = [x for x in range(len(t['Filename'])) ]

    for feat, color in zip(t.iloc[:, 1:], colors):
        p.line(t['Filename'], t[feat], line_width=2, color=color, legend_label=f"{feat} {name}")
        p.legend.location = "top_left"
        p.legend.click_policy="mute"
        p.xaxis.major_label_orientation = "vertical"
    show(p)

In [48]:

def plot_charts():
    for key, values in columns_to_plot.items():
        ncols, features = values.values()

        df = all_kinds_df[key]

        # questo array inizialmente contiene una lista vuota
        # LIST [ LIST[] ]
        # in ogni sottolista verranno aggiunte le figure (bokeh.figure) da stampare.
        # Una volta raggiunta la fine della riga (ncols) l'ultima lista viene convertita
        # in bokeh.row
        # Risultato finale:
        #   LIST [ bokeh.row(p1,p2,p3), bokeh.row(p4, p5, p6) ]
        rows = []
        rows.append([])

        unique_names = df['name'].unique()

        display(Markdown(f'## {key.upper()}'))

        axes_i = 0 
        axes_j = 0
        for feature in features:
            # palette di colori da usare per disegnare le singole linee
            colors = itertools.cycle(palette) 

            p = figure(x_axis_type='datetime')
            p.title = feature

            for color, name in zip(colors, unique_names): # ad ogni feature assegna un colore
                tmp = df[df['name'] == name].copy()
                x = pd.to_datetime(tmp['timestamp'])
                
                p.line(x, tmp[feature], line_width=2, color=color, legend_label=f"{feature} {name}")
                p.legend.location = "top_left"
                p.legend.click_policy="mute" # al click la linea diventa quasi trasparente

            # il plot del singolo grafo è finito
            rows[axes_i].append(p)

            # aggiunge una nuova riga quando serve
            axes_j+= 1
            if axes_j >= ncols:
                axes_i += 1
                axes_j = 0
                rows.append([])

        # se l'ultimo elemento è vuoto lo elimina
        if rows[-1] == []:
            rows = rows[:-1]

        rows = [row(elem) for elem in rows]

        show(column(*rows))
    